In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# import libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df = pd.read_csv("/kaggle/input/ab-testing-practice/ab_testing.csv")

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.groupby("Group")[["Page Views"]].mean()

In [ ]:
df.groupby("Page Views")[["Time Spent"]].mean()

In [ ]:
df.groupby("Group")[["Time Spent"]].mean()

In [ ]:
df

In [ ]:
df["Location"].unique()

In [ ]:
df["Page Views"].unique()

In [ ]:
df["Device"].value_counts()

In [ ]:
df["Page Views"].value_counts()

In [ ]:
df.info()

# EDA

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt

msno.matrix(df)
plt.show

In [ ]:
group_means = df.groupby("Device")["Page Views"].sum()

plt.bar(group_means.index, group_means.values, color=["blue", "green"])
plt.title("Aygıtlara (Device) Göre Toplam Sayfa Görüntüleme")
plt.xlabel("Device")
plt.ylabel("Toplam Sayfa Görüntüleme")
plt.show()

In [ ]:
# Location dağılımı

location_counts = df["Location"].value_counts()

plt.pie(location_counts, labels=location_counts.index, autopct="%1.1f%%", startangle=140, colors=["green", "lightblue"])
plt.title("Location Dağılımı")
plt.show()

In [ ]:
plt.hist(df["Time Spent"], bins=50, color="purple", edgecolor="black")

plt.title("Ziyaret Süresi Dağılımı")
plt.xlabel("Ziyaret Süresi (saniye)")
plt.ylabel("Frekans")
plt.show()

In [ ]:
location_counts = df["Location"].value_counts()

plt.bar(location_counts.index, location_counts.values, color="teal")

plt.title("Lokasyonlara Göre Kullanıcı Sayısı")
plt.xlabel("Lokasyon")
plt.ylabel("Kullanıcı Sayısı")
plt.xticks(rotation=45)
plt.show()

In [ ]:
df

In [ ]:
import seaborn as sns
plt.figure(figsize=(12, 6))
sns.boxplot(x='Location', y='Time Spent', data=df)

plt.title('Ziyaret Süresine Göre Konum Dağılımı')
plt.xlabel('Konum')
plt.ylabel('Ziyaret Süresi (saniye)')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# Data preprocessing

In [ ]:
from sklearn import preprocessing 

In [ ]:
label_encoder = preprocessing.LabelEncoder() 
  
 
df['Group']= label_encoder.fit_transform(df['Group']) 

In [ ]:
label_encoder = preprocessing.LabelEncoder() 
  
 
df['Conversion']= label_encoder.fit_transform(df['Conversion']) 

In [ ]:
label_encoder = preprocessing.LabelEncoder() 
  
 
df['Location']= label_encoder.fit_transform(df['Location']) 

In [ ]:
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['Device']= label_encoder.fit_transform(df['Device']) 

In [ ]:
df.drop(['User ID'], axis='columns', inplace=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['Page Views', 'Time Spent']] = scaler.fit_transform(df[['Page Views', 'Time Spent']])

In [ ]:
# Separate features and target
X = df.drop(['Conversion'], axis=1)  # Drop irrelevant columns
y = df['Conversion'] 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Support Vector Machine": SVC(probability=True),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "XGBoost": XGBClassifier(eval_metric='mlogloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42)
}


# Train and Evaluate Models

In [ ]:
# Store evaluation metrics
results = []

for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
    
    # Evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    roc_auc = roc_auc_score(y_test, y_pred_prob) if y_pred_prob is not None else None
    
    # Save results
    results.append({
        "Model": name,
        "Accuracy": accuracy,
        "Precision": report["1"]["precision"],
        "Recall": report["1"]["recall"],
        "F1-Score": report["1"]["f1-score"],
        "ROC-AUC": roc_auc
    })


# Visualize Results

In [ ]:
# Create a DataFrame of results
results_df = pd.DataFrame(results)

# Plot Accuracy, Precision, Recall, F1-Score, and ROC-AUC
metrics = ["Accuracy", "Precision", "Recall", "F1-Score", "ROC-AUC"]
for metric in metrics:
    plt.figure(figsize=(10, 6))
    sns.barplot(x="Model", y=metric, data=results_df.sort_values(by=metric, ascending=False), palette="viridis")
    plt.title(f"{metric} by Model", fontsize=16)
    plt.xticks(rotation=45)
    plt.ylabel(metric)
    plt.xlabel("Model")
    plt.show()


# Compare the Models

In [ ]:
# Display sorted results
print(results_df.sort_values(by="Accuracy", ascending=False))
